Here data is accessed from HistoricalElectricityWeather table in database and then model is trainned and updated. This trainning is done once a month

In [4]:
#import required libraries
import pandas as pd
import numpy as np
import joblib
import sqlite3

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [6]:
# Connect to the SQLite database
conn = sqlite3.connect("../prisma/db.sqlite")
# Query recent new feature with target dataset
query = "SELECT * FROM HistoricalElectricityWeather  WHERE price IS NOT NULL"
data = pd.read_sql_query(query, conn)

In [15]:
# Select specific columns using .filter
X = data.filter(['temperature', 'precipitation', 'weatherCodeId', 'cloudCover', 'windSpeed10m', 'shortwaveRadiation'])
y=data.filter(['price'])
print(X.shape)
print(y.shape)

(21478, 6)
(21478, 1)


In [16]:
# Normalize the features
# Load the scalers from disk
scaler_X = joblib.load('scaler_X.pkl')
scaler_y = joblib.load('scaler_y.pkl')
X_normalized = scaler_X.transform(X)
y_normalize = scaler_y.transform(y.values.reshape(-1, 1))

/home/ashagaire/Desktop/LearnIT and coding with Finnish Language/Mittwatt/electricity-shoc/mittwatt/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [18]:
# Load the model
model = load_model('updated_electricity_price_model.h5')

/home/ashagaire/Desktop/LearnIT and coding with Finnish Language/Mittwatt/electricity-shoc/mittwatt/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-27 14:50:47.611878: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-27 14:50:47.744467: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at http

In [19]:
# Step 3: Recompile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_absolute_error'])

In [20]:
model.fit(X_normalized, y_normalize, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/100


I0000 00:00:1719489058.469612    9920 service.cc:145] XLA service 0x7e196c002680 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719489058.469661    9920 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 Ti, Compute Capability 7.5
2024-06-27 14:50:58.543429: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-27 14:50:58.863090: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 59/537 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0022 - mean_absolute_error: 0.0342

I0000 00:00:1719489064.085689    9920 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


537/537 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - loss: 0.0021 - mean_absolute_error: 0.0328 - val_loss: 0.0038 - val_mean_absolute_error: 0.0304
Epoch 2/100
537/537 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0022 - mean_absolute_error: 0.0335 - val_loss: 0.0068 - val_mean_absolute_error: 0.0353
Epoch 3/100
537/537 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0021 - mean_absolute_error: 0.0331 - val_loss: 0.0060 - val_mean_absolute_error: 0.0340
Epoch 4/100
537/537 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0021 - mean_absolute_error: 0.0329 - val_loss: 0.0038 - val_mean_absolute_error: 0.0329
Epoch 5/100
537/537 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0021 - mean_absolute_error: 0.0328 - val_loss: 0.0047 - val_mean_absolute_error: 0.0310
Epoch 6/100
537/537 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0021 - mean_absolute_error: 0.0328 - val_loss: 0.0033 - val_mean_absolute_error: 0.0313
Epoch 7/100
537/537 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0021 - mean_absolute_error: 0.0325 - val_

In [21]:
# Step 4: Save the updated model
model.save('updated_electricity_price_model.h5')

In [22]:
# Close the connection
conn.close()